In [ ]:
from ml4cvd.arguments import _get_tmap
from ml4cvd.tensor_generators import get_test_train_valid_paths
from ml4cvd.tensor_maps_by_hand import TMAPS

train_paths, valid_paths, test_paths = get_test_train_valid_paths('/mnt/disks/ecg-rest-37k-tensors/2019-11-04/', 0.1, 0.1, 10, None)
all_paths = train_paths+valid_paths+test_paths

In [ ]:
import h5py
import numpy as np
import time
df_dic = {'full_path': [], 'patient_id': [], 'ecg_rest_text': [], 
          'rr': [], 'pp': [], 'pr': [], 'qrs': [],  'qt': [], 'qtc': [], 'paxis': [], 'raxis': [], 'taxis': []
         }
cnt = 0
for path in all_paths:
    hd5 = h5py.File(path, 'r')
    df_dic['full_path'].append(path)
    df_dic['patient_id'].append(path.split('/')[-1].split('.hd5')[0])
    time_start = time.time()
    df_dic['ecg_rest_text'].append(hd5['ecg_rest_text'][0])
    time_string = time.time()
    try:
        df_dic['rr'].append(hd5['continuous']['RRInterval'])    
    except:
        print(f"{df_dic['patient_id'][-1]} does not have RR measure tensorized")
        df_dic['rr'].append(np.nan)
    time_rr = time.time()
    try:
        df_dic['pp'].append(hd5['continuous']['PPInterval'])
    except:
        print(f"{df_dic['patient_id'][-1]} does not have PP measure tensorized")
        df_dic['pp'].append(np.nan)
    time_pp = time.time()
    try:
        df_dic['pr'].append(hd5['continuous']['PQInterval'])
        df_dic['paxis'].append(hd5['continuous']['PAxis'])
    except:
        print(f"{df_dic['patient_id'][-1]} does not have P measures tensorized")
        df_dic['pr'].append(np.nan)
        df_dic['paxis'].append(np.nan)
    time_pr = time.time()
    try:
        df_dic['qrs'].append(hd5['continuous']['QRSDuration'])
        df_dic['qt'].append(hd5['continuous']['QTInterval'])
        df_dic['qtc'].append(hd5['continuous']['QTCInterval'])
    except:
        print(f"{df_dic['patient_id'][-1]} does not have QRS measures tensorized")
        df_dic['qrs'].append(np.nan)
        df_dic['qt'].append(np.nan)
        df_dic['qtc'].append(np.nan)
    time_qt = time.time()
    try:
        df_dic['raxis'].append(hd5['continuous']['RAxis'])
        df_dic['taxis'].append(hd5['continuous']['TAxis'])
    except:
        print(f"{df_dic['patient_id'][-1]} does not have R and T axis measures tensorized")
        df_dic['raxis'].append(np.nan)
        df_dic['taxis'].append(np.nan)
    time_axis = time.time()
    hd5.close()
    cnt += 1
    #print(time_string-time_start, time_rr-time_start, time_pp-time_start, time_pr-time_start, time_qt-time_start, time_axis-time_start)
    #if cnt == 1000: break

In [ ]:
import pandas as pd
df = pd.DataFrame(df_dic)

In [ ]:
df.to_csv('/home/pdiachil/hierarchical_intervals.csv')

In [ ]:
class ecg_class:
    def __init__(self, ecg_class_id, name=None):
        self.id = ecg_class_id
        self.name = name
        self.children = []
        self.children_ids = []
        self.matching_strings = []
        self.ncounts = 0
        self.patients = [] 
        self.class_df = None
    
    def __getitem__(self, key):        
        if key not in self.children_ids:
            raise KeyError(f'Class {key} is not a member of {self.id}')
        idx = self.children_ids.index(key)
        return self.children[idx]

    def __str__(self):
        return f'{self.name}: {self.ncounts} ({self.ncounts / 37623.0 * 100.0 :.2f}%)'

    def __iter__(self):
        for child_id in self.children_ids:
            yield child_id

    def __contains__(self, key):        
        return key in self.children_ids
    
    def add_child(self, ecg_class):
        if ecg_class.id in self.children_ids:
            print("WARNING: class with same id already among subclasses. Skipping")
        else:
            self.children.append(ecg_class)
            self.children_ids.append(ecg_class.id)
            
    def add_matching_string(self, new_string):
        self.matching_strings.append(new_string)
        self.matching_strings = list(set(self.matching_strings))
        
    def make_children_exclusive(self, df):
        if not self.children:
            raise KeyError(f'Class {self.name} does not have subclasses. Use update_class_df instead')
        for child in self.children:
            if child.class_df is None:
                child.update_class_df(df)
        class_df = self.children[0].class_df
        for c2 in self.children[1:]:                
            inters = class_df.index.intersection(c2.class_df.index)
            c2.class_df = c2.class_df.drop(inters)
            c2.ncounts = len(c2.class_df)
            class_df = pd.concat([class_df, c2.class_df])
        self.class_df = class_df
        self.ncounts = len(class_df)

    def make_child_exclusive(self, child_id, df):
        if child_id not in self.children_ids:
            raise KeyError(f'Class {key} is not a member of {self.id}')
        child_idx = self.children_ids.index(child_id)
        child = self.children.pop(child_idx)
        self.children.append(child)
        class_df = self.children[0].class_df        
        for c2 in self.children[1:]:
            inters = class_df.index.intersection(c2.class_df.index)
            c2_class_df = c2.class_df.drop(inters)
            class_df = pd.concat([class_df, c2_class_df])
        c2.class_df = c2_class_df
        c2.ncounts = len(c2_class_df)    
        
    def update_children_df(self, df):
        if not self.children:
            raise KeyError(f'Class {self.name} does not have subclasses. Use update_class_df instead')
        for child in self.children:
            if child.class_df is None:
                child.update_class_df(df)
        class_df = self.children[0].class_df
        for c2 in self.children[1:]:
            inters = class_df.index.intersection(c2.class_df.index)
            c2_class_df = c2.class_df.drop(inters)
            class_df = pd.concat([class_df, c2_class_df])
        self.class_df = class_df            
        self.ncounts = len(class_df)
            
    def update_class_df(self, df):
        if not self.matching_strings:
            raise KeyError(f'Class {self.name} does not have matching string pattern to extract patients')
        class_df = df[df['ecg_rest_text'].str.count(self.matching_strings[0]) > 0.5]
        for matching_string in self.matching_strings[1:]:
            df2 = df[df['ecg_rest_text'].str.count(matching_string) > 0.5]
            inters = class_df.index.intersection(df2.index)
            df2 = df2.drop(inters)
            class_df = pd.concat([class_df, df2])
        self.class_df = class_df
        self.ncounts = len(class_df)   

In [ ]:
classification = ecg_class('class', 'Classification')

In [ ]:
rhythm = ecg_class('rhythm', 'Rhythm')
classification.add_child(rhythm)

In [ ]:
sinus_rhythm = ecg_class('sinus_rhythm', 'Sinus rhythm')
classification['rhythm'].add_child(sinus_rhythm)
sinus_rhythm_no_arr = ecg_class('no_arr', 'Sinus rhythm no arrhythmia')
sinus_rhythm_yes_arr = ecg_class('yes_arr', 'Sinus rhythm with arrhythmia')
classification['rhythm']['sinus_rhythm'].add_child(sinus_rhythm_no_arr)
classification['rhythm']['sinus_rhythm'].add_child(sinus_rhythm_yes_arr)
classification['rhythm']['sinus_rhythm']['no_arr'].add_matching_string(r'[sS]inus rhythm')
classification['rhythm']['sinus_rhythm']['yes_arr'].add_matching_string(r'[sS]inus bradycardia')
classification['rhythm']['sinus_rhythm']['yes_arr'].add_matching_string(r'[sS]inus tachycardia')
classification['rhythm']['sinus_rhythm']['yes_arr'].add_matching_string(r'[sS]inus arrhythmia')
classification['rhythm']['sinus_rhythm'].make_children_exclusive(df)
print(classification['rhythm']['sinus_rhythm']['yes_arr'])
print(classification['rhythm']['sinus_rhythm']['no_arr'])

In [ ]:
ectopic_atrial = ecg_class('ectopic_atrial', 'Ectopic atrial rhythm')
classification['rhythm'].add_child(ectopic_atrial)
classification['rhythm']['ectopic_atrial'].add_matching_string(r'[Ee]ctopic atrial')
classification['rhythm']['ectopic_atrial'].add_matching_string(r'[Uu]nusual P axis possible ectopic atrial tachycardia')
classification['rhythm']['ectopic_atrial'].add_matching_string(r'[Uu]nusual P axis')
classification['rhythm']['ectopic_atrial'].update_class_df(df)
print(classification['rhythm']['ectopic_atrial'])

In [ ]:
atrial_fibrillation = ecg_class('atrial_fibrillation', 'Atrial fibrillation')
classification['rhythm'].add_child(atrial_fibrillation)
classification['rhythm']['atrial_fibrillation'].add_matching_string(r'Atrial fibrillation')
classification['rhythm']['atrial_fibrillation'].update_class_df(df)
print(classification['rhythm']['atrial_fibrillation'])

In [ ]:
atrial_flutter = ecg_class('atrial_flutter', 'Atrial flutter')
classification['rhythm'].add_child(atrial_flutter)
atrial_flutter_fixed = ecg_class('fixed', 'Atrial flutter with fixed block')
atrial_flutter_variable = ecg_class('variable', 'Atrial flutter with variable block')
atrial_flutter_unspec = ecg_class('unspec', 'Atrial flutter with unspecified block')
classification['rhythm']['atrial_flutter'].add_child(atrial_flutter_fixed)
classification['rhythm']['atrial_flutter'].add_child(atrial_flutter_variable)
classification['rhythm']['atrial_flutter'].add_child(atrial_flutter_unspec)
classification['rhythm']['atrial_flutter']['fixed'].add_matching_string(r'[aA]trial flutter with [0-9]:1 AV conduction')
classification['rhythm']['atrial_flutter']['variable'].add_matching_string(r'[aA]trial flutter with variable AV block')
classification['rhythm']['atrial_flutter']['unspec'].add_matching_string('r[aA]trial flutter')
classification['rhythm']['atrial_flutter'].make_children_exclusive(df)
print(classification['rhythm']['atrial_flutter'])
print(classification['rhythm']['atrial_flutter']['fixed'])
print(classification['rhythm']['atrial_flutter']['variable'])
print(classification['rhythm']['atrial_flutter']['unspec'])

In [ ]:
undetermined_rhythm = ecg_class('undetermined', 'Undetermined rhythm')
classification['rhythm'].add_child(undetermined_rhythm)
classification['rhythm']['undetermined'].add_matching_string('Undetermined rhythm')
classification['rhythm']['undetermined'].update_class_df(df)
print(classification['rhythm']['undetermined'])

In [ ]:
junctional = ecg_class('junctional_rhythm', 'Junctional Rhythm')
classification.add_child(junctional)
classification['junctional_rhythm'].add_matching_string(r'[jJ]unctional rhythm')
classification['junctional_rhythm'].add_matching_string(r'[jJ]unctional pacemaker')
classification['junctional_rhythm'].add_matching_string(r'[jJ]unctional bradycardia')
classification['junctional_rhythm'].add_matching_string(r'[jJ]unctional escape complexes')
classification['junctional_rhythm'].update_class_df(df)
print(classification['junctional_rhythm'])

In [ ]:
ventricular = ecg_class('ventricular_rhythm', 'Ventricular Rhythm')
classification.add_child(ventricular)
classification['ventricular_rhythm'].add_matching_string(r'[vV]entricular escape complexes')
classification['ventricular_rhythm'].update_class_df(df)
print(classification['ventricular_rhythm'])

In [ ]:
av_block = ecg_class('av_block', 'AV Block')
classification.add_child(av_block)
first_deg_block = ecg_class('first_deg', '1st degree AV Block')
second_deg_block = ecg_class('second_deg', '2nd degree AV Block')
third_deg_block = ecg_class('third_deg', '3rd degree AV Block')
dissoc_block = ecg_class('dissociation', 'AV dissociation')
classification['av_block'].add_child(first_deg_block)
classification['av_block']['first_deg'].add_matching_string('1st degree AV block')
classification['av_block'].add_child(second_deg_block)
classification['av_block']['second_deg'].add_matching_string('2nd degree AV block')
mobitz_I  = ecg_class('mobitz_I', '2nd degree AV block (Mobitz I)')
mobitz_II = ecg_class('mobitz_II', '2nd degree AV block (Mobitz II)')
unspec_av_block = ecg_class('unspecified', '2nd degree AV block (unspecified)')
classification['av_block']['second_deg'].add_child(mobitz_I)
classification['av_block']['second_deg'].add_child(mobitz_II)
classification['av_block']['second_deg'].add_child(unspec_av_block)
classification['av_block']['second_deg']['mobitz_I'].add_matching_string('Mobitz I')
classification['av_block']['second_deg']['mobitz_II'].add_matching_string('Mobitz II')
classification['av_block']['second_deg']['unspecified'].add_matching_string('2nd degree AV block')
classification['av_block'].add_child(third_deg_block)
classification['av_block']['third_deg'].add_matching_string('3rd degree AV block')
classification['av_block'].add_child(dissoc_block)
classification['av_block']['dissociation'].add_matching_string('AV dissociation')
classification['av_block']['second_deg'].make_children_exclusive(df)
classification['av_block'].make_children_exclusive(df)
print(classification['av_block'])
print(classification['av_block']['first_deg'])
print(classification['av_block']['second_deg'])
print(classification['av_block']['second_deg']['mobitz_I'])
print(classification['av_block']['second_deg']['mobitz_II'])
print(classification['av_block']['second_deg']['unspecified'])
print(classification['av_block']['third_deg'])
print(classification['av_block']['dissociation'])

In [ ]:
# Subclasses of BBB can all occur at the same time
bbb = ecg_class('bbb', 'Bundle branch block')
classification.add_child(bbb)
bbb_left = ecg_class('left', 'Left bundle branch block')
bbb_right = ecg_class('right', 'Right bundle branch block')
bbb_ivcd = ecg_class('ivcd', 'Intraventricular conduction delay')
bbb_la_fascicular = ecg_class('la_fascicular', 'Left anterior fascicular block')
bbb_lp_fascicular = ecg_class('lp_fascicular', 'Left posterior fascicular block')
classification['bbb'].add_child(bbb_left)
classification['bbb'].add_child(bbb_right)
classification['bbb'].add_child(bbb_ivcd)
classification['bbb'].add_child(bbb_la_fascicular)
classification['bbb'].add_child(bbb_lp_fascicular)
classification['bbb']['ivcd'].add_matching_string(r'intraventricular block')
classification['bbb']['ivcd'].add_matching_string(r'intraventricular conduction delay')
classification['bbb']['la_fascicular'].add_matching_string(r'[lL]eft anterior fascicular block')
classification['bbb']['lp_fascicular'].add_matching_string(r'[lL]eft posterior fascicular block')
classification['bbb']['right'].add_matching_string(r'[rR]ight bundle branch block')
bbb_left_complete = ecg_class('complete', 'Complete left bundle branch block')
bbb_left_incomplete = ecg_class('incomplete', 'Incomplete left bundle branch block')
bbb_right_complete = ecg_class('complete', 'Complete right bundle branch block')
bbb_right_incomplete = ecg_class('incomplete', 'Incomplete right bundle branch block')
classification['bbb']['left'].add_child(bbb_left_incomplete)
classification['bbb']['left'].add_child(bbb_left_complete)
classification['bbb']['right'].add_child(bbb_right_complete)
classification['bbb']['right'].add_child(bbb_right_incomplete)
classification['bbb']['left']['incomplete'].add_matching_string(r'[Ii]ncomplete left bundle branch block')
classification['bbb']['left']['complete'].add_matching_string(r'Left bundle branch block')
classification['bbb']['right']['incomplete'].add_matching_string(r'[Ii]ncomplete right bundle branch block')
classification['bbb']['right']['complete'].add_matching_string(r'Right bundle branch block')
classification['bbb']['left'].make_children_exclusive(df)
classification['bbb']['right'].make_children_exclusive(df)
classification['bbb'].update_children_df(df)
print(classification['bbb'])
print(classification['bbb']['left'])
print(classification['bbb']['left']['complete'])
print(classification['bbb']['left']['incomplete'])
print(classification['bbb']['la_fascicular'])
print(classification['bbb']['lp_fascicular'])
print(classification['bbb']['right'])
print(classification['bbb']['right']['complete'])
print(classification['bbb']['right']['incomplete'])
print(classification['bbb']['ivcd'])

In [ ]:
ectopy = ecg_class('ectopy', 'Ectopy')
classification.add_child(ectopy)
ectopy_pacs = ecg_class('pacs', 'Premature atrial complexes')
ectopy_pvcs = ecg_class('pvcs', 'Premature ventricular complexes')
classification['ectopy'].add_child(ectopy_pacs)
classification['ectopy'].add_child(ectopy_pvcs)
ectopy_pacs_isolated = ecg_class('isolated', 'Isolated premature atrial contractions')
ectopy_pacs_bigeminy = ecg_class('atrial_bigeminy', 'Atrial bigeminy')
ectopy_pacs_trigeminy = ecg_class('atrial_trigeminy', 'Atrial trigeminy')
classification['ectopy']['pacs'].add_child(ectopy_pacs_bigeminy)
classification['ectopy']['pacs'].add_child(ectopy_pacs_trigeminy)
classification['ectopy']['pacs'].add_child(ectopy_pacs_isolated)
classification['ectopy']['pacs']['atrial_bigeminy'].add_matching_string(r'atrial complexes in a pattern of bigeminy')
classification['ectopy']['pacs']['atrial_trigeminy'].add_matching_string(r'atrial complexes in a pattern of trigeminy')
classification['ectopy']['pacs']['isolated'].add_matching_string(r'[pP]remature atrial complexes')
classification['ectopy']['pacs'].make_children_exclusive(df)
ectopy_pvcs_isolated = ecg_class('isolated', 'Isolated premature ventricular contractions')
ectopy_pvcs_bigeminy = ecg_class('ventricular_bigeminy', 'Ventricular bigeminy')
ectopy_pvcs_trigeminy = ecg_class('ventricular_trigeminy', 'Ventricular trigeminy')
classification['ectopy']['pvcs'].add_child(ectopy_pvcs_bigeminy)
classification['ectopy']['pvcs'].add_child(ectopy_pvcs_trigeminy)
classification['ectopy']['pvcs'].add_child(ectopy_pvcs_isolated)
classification['ectopy']['pvcs']['ventricular_bigeminy'].add_matching_string(r'ventricular complexes in a pattern of bigeminy')
classification['ectopy']['pvcs']['ventricular_trigeminy'].add_matching_string(r'ventricular complexes in a pattern of trigeminy')
classification['ectopy']['pvcs']['isolated'].add_matching_string(r'[pP]remature ventricular complexes')
classification['ectopy']['pvcs'].make_children_exclusive(df)
classification['ectopy'].update_children_df(df)
print(classification['ectopy'])
print(classification['ectopy']['pacs'])
print(classification['ectopy']['pacs']['atrial_bigeminy'])
print(classification['ectopy']['pacs']['atrial_trigeminy'])
print(classification['ectopy']['pacs']['isolated'])
print(classification['ectopy']['pvcs'])
print(classification['ectopy']['pvcs']['ventricular_bigeminy'])
print(classification['ectopy']['pvcs']['ventricular_trigeminy'])
print(classification['ectopy']['pvcs']['isolated'])


In [ ]:
# Structural morphologies
structural = ecg_class('structural', 'Structural morphologies')
classification.add_child(structural)
lae = ecg_class('lae', 'Left atrial enlargement')
rae = ecg_class('rae', 'Right atrial enlargement')
classification['structural'].add_child(lae)
classification['structural'].add_child(rae)
classification['structural']['lae'].add_matching_string('[lL]eft atrial enlargement')
classification['structural']['lae'].add_matching_string('[bB]iatrial enlargement')
classification['structural']['rae'].add_matching_string('[rR]ight atrial enlargement')
classification['structural']['rae'].add_matching_string('[bB]iatrial enlargement')
old_infarct = ecg_class('old_infarct', 'Old myocardial infarct')
classification['structural'].add_child(old_infarct)
old_infarct_septal = ecg_class('septal', 'Old myocardial infarct (septal)')
old_infarct_anterior = ecg_class('anterior', 'Old myocardial infarct (anterior)')
old_infarct_inferior = ecg_class('inferior', 'Old myocardial infarct (inferior)')
old_infarct_lateral = ecg_class('lateral', 'Old myocardial infarct (lateral)')
old_infarct_posterior = ecg_class('posterior', 'Old myocardial infarct (posterior)')
old_infarct_unspec = ecg_class('unspec', 'Old myocardial infarct')
classification['structural']['old_infarct'].add_child(old_infarct_septal)
classification['structural']['old_infarct'].add_child(old_infarct_anterior)
classification['structural']['old_infarct'].add_child(old_infarct_inferior)
classification['structural']['old_infarct'].add_child(old_infarct_lateral)
classification['structural']['old_infarct'].add_child(old_infarct_posterior)
classification['structural']['old_infarct'].add_child(old_infarct_unspec)
classification['structural']['old_infarct']['septal'].add_matching_string('[sS]eptal infarct age undetermined')
classification['structural']['old_infarct']['septal'].add_matching_string('[aA]nteroseptal infarct age undetermined')
classification['structural']['old_infarct']['anterior'].add_matching_string('[sA]nterior infarct age undetermined')
classification['structural']['old_infarct']['anterior'].add_matching_string('[sA]nteroseptal infarct age undetermined')
classification['structural']['old_infarct']['anterior'].add_matching_string('[sA]nterolateral infarct age undetermined')
classification['structural']['old_infarct']['inferior'].add_matching_string('[iI]nferior infarct age undetermined')
classification['structural']['old_infarct']['inferior'].add_matching_string('[iI]nferior-posterior infarct age undetermined')
classification['structural']['old_infarct']['posterior'].add_matching_string('[iI]nferior-posterior infarct age undetermined')
classification['structural']['old_infarct']['posterior'].add_matching_string('[pP]osterior infarct age undetermined')
classification['structural']['old_infarct']['lateral'].add_matching_string('[sA]nterolateral infarct age undetermined')
classification['structural']['old_infarct']['lateral'].add_matching_string('[lL]ateral infarct age undetermined')
classification['structural']['old_infarct']['unspec'].add_matching_string('[iI]nfarct age undetermined')
classification['structural']['old_infarct'].update_children_df(df)
classification['structural']['old_infarct'].make_child_exclusive('unspec', df)
classification['structural'].update_children_df(df)
print(classification['structural']['lae'])
print(classification['structural']['rae'])
print(classification['structural']['old_infarct'])
print(classification['structural']['old_infarct']['septal'])
print(classification['structural']['old_infarct']['anterior'])
print(classification['structural']['old_infarct']['inferior'])
print(classification['structural']['old_infarct']['posterior'])
print(classification['structural']['old_infarct']['lateral'])
print(classification['structural']['old_infarct']['unspec'])



In [ ]:
acute_infarct = ecg_class('acute_infarct', 'Acute myocardial infarct')
classification['structural'].add_child(acute_infarct)
acute_infarct_septal = ecg_class('septal', 'Acute myocardial infarct (septal)')
acute_infarct_anterior = ecg_class('anterior', 'Acute myocardial infarct (anterior)')
acute_infarct_inferior = ecg_class('inferior', 'Acute myocardial infarct (inferior)')
acute_infarct_lateral = ecg_class('lateral', 'Acute myocardial infarct (lateral)')
acute_infarct_posterior = ecg_class('posterior', 'Acute myocardial infarct (posterior)')
acute_infarct_unspec = ecg_class('unspec', 'Acute myocardial infarct')
classification['structural']['acute_infarct'].add_child(acute_infarct_septal)
classification['structural']['acute_infarct'].add_child(acute_infarct_anterior)
classification['structural']['acute_infarct'].add_child(acute_infarct_inferior)
classification['structural']['acute_infarct'].add_child(acute_infarct_lateral)
classification['structural']['acute_infarct'].add_child(acute_infarct_posterior)
classification['structural']['acute_infarct'].add_child(acute_infarct_unspec)
classification['structural']['acute_infarct']['septal'].add_matching_string('[sS]eptal infarct possibly acute')
classification['structural']['acute_infarct']['septal'].add_matching_string('[sS]eptal injury pattern')
classification['structural']['acute_infarct']['septal'].add_matching_string('[aA]nteroseptal infarct possibly acute')
classification['structural']['acute_infarct']['anterior'].add_matching_string('ST elevation consider anterior injury or acute infarct')
classification['structural']['acute_infarct']['anterior'].add_matching_string('ST elevation consider anterolateral injury or acute infarct')
classification['structural']['acute_infarct']['anterior'].add_matching_string('[aA]nterior injury pattern')
classification['structural']['acute_infarct']['anterior'].add_matching_string('[aA]nterior infarct possibly acute')
classification['structural']['acute_infarct']['anterior'].add_matching_string('[Aa]nteroseptal infarct possibly acute')
classification['structural']['acute_infarct']['inferior'].add_matching_string('ST elevation consider inferior injury or acute infarct')
classification['structural']['acute_infarct']['inferior'].add_matching_string('ST elevation consider inferolateral injury or acute infarct')
classification['structural']['acute_infarct']['inferior'].add_matching_string('[aA]cute inferior infarct')
classification['structural']['acute_infarct']['inferior'].add_matching_string('[iI]nferior injury pattern')
classification['structural']['acute_infarct']['inferior'].add_matching_string('[iI]nferior infarct possibly acute')
classification['structural']['acute_infarct']['inferior'].add_matching_string('[iI]nferior-posterior infarct possibly acute')
classification['structural']['acute_infarct']['posterior'].add_matching_string('[pP]osterior injury pattern')
classification['structural']['acute_infarct']['posterior'].add_matching_string('[pP]osterior infarct possibly acute')
classification['structural']['acute_infarct']['posterior'].add_matching_string('[iI]nferior-posterior infarct possibly acute')
classification['structural']['acute_infarct']['lateral'].add_matching_string('[lL]ateral injury pattern')
classification['structural']['acute_infarct']['lateral'].add_matching_string('[lL]ateral infarct possibly acute')
classification['structural']['acute_infarct']['lateral'].add_matching_string('ST elevation consider lateral injury or acute infarct')
classification['structural']['acute_infarct']['lateral'].add_matching_string('ST elevation consider inferolateral injury or acute infarct')
classification['structural']['acute_infarct']['lateral'].add_matching_string('ST elevation consider anterolateral injury or acute infarct')
classification['structural']['acute_infarct']['lateral'].add_matching_string('[sA]nterolateral infarct possibly acute')
classification['structural']['acute_infarct']['lateral'].add_matching_string('[lL]ateral infarct age possibly acute')
classification['structural']['acute_infarct']['unspec'].add_matching_string('[aA][cC][uU][tT][eE] [mM][iI]')
classification['structural']['acute_infarct'].update_children_df(df)
classification['structural']['acute_infarct'].make_child_exclusive('unspec', df)
classification['structural'].update_children_df(df)
print(classification['structural']['acute_infarct'])
print(classification['structural']['acute_infarct']['septal'])
print(classification['structural']['acute_infarct']['anterior'])
print(classification['structural']['acute_infarct']['inferior'])
print(classification['structural']['acute_infarct']['posterior'])
print(classification['structural']['acute_infarct']['lateral'])
print(classification['structural']['acute_infarct']['unspec'])

In [ ]:
st_depression = ecg_class('st_depression', 'ST Depression')
classification['structural'].add_child(st_depression)
st_depression_unspec = ecg_class('unspec', 'Unspecified ST depression')
classification['structural']['st_depression'].add_child(st_depression_unspec)
classification['structural']['st_depression']['unspec'].add_matching_string('ST depression probably abnormal')
classification['structural']['st_depression']['unspec'].add_matching_string('ST depression probably normal')
classification['structural']['st_depression'].update_children_df(df)
print(classification['structural']['st_depression'])
print(classification['structural']['st_depression']['unspec'])

In [ ]:
st_abnormal = ecg_class('st_abnormality', 'ST segment abnormality')
classification['structural'].add_child(st_abnormal)
st_abnormal_septal = ecg_class('septal', 'ST abnormality localized septally')
st_abnormal_lateral = ecg_class('lateral', 'ST abnormality localized laterally')
st_abnormal_inferior = ecg_class('inferior', 'ST abnormality localized inferiorly')
st_abnormal_anterior = ecg_class('anterior', 'ST abnormality localized anteriorly')
st_abnormal_posterior = ecg_class('posterior', 'ST abnormality localized posteriorly')
st_abnormal_unspec = ecg_class('unspec', 'Unspecified ST segment abnormality')
classification['structural']['st_abnormality'].add_child(st_abnormal_septal)
classification['structural']['st_abnormality'].add_child(st_abnormal_lateral)
classification['structural']['st_abnormality'].add_child(st_abnormal_inferior)
classification['structural']['st_abnormality'].add_child(st_abnormal_anterior)
classification['structural']['st_abnormality'].add_child(st_abnormal_posterior)
classification['structural']['st_abnormality'].add_child(st_abnormal_unspec)
classification['structural']['st_abnormality']['septal'].add_matching_string('ST abnormality consider septal ischemia')
classification['structural']['st_abnormality']['septal'].add_matching_string('ST and T wave abnormality consider septal ischemia')
classification['structural']['st_abnormality']['lateral'].add_matching_string('ST abnormality consider lateral ischemia')
classification['structural']['st_abnormality']['lateral'].add_matching_string('ST abnormality consider anterolateral ischemia')
classification['structural']['st_abnormality']['lateral'].add_matching_string('ST abnormality consider inferolateral ischemia')
classification['structural']['st_abnormality']['lateral'].add_matching_string('ST and T wave abnormality consider lateral ischemia')
classification['structural']['st_abnormality']['lateral'].add_matching_string('ST and T wave abnormality consider anterolateral ischemia')
classification['structural']['st_abnormality']['lateral'].add_matching_string('ST and T wave abnormality consider inferolateral ischemia')
classification['structural']['st_abnormality']['inferior'].add_matching_string('ST abnormality consider inferior ischemia')
classification['structural']['st_abnormality']['inferior'].add_matching_string('ST abnormality consider inferolateral ischemia')
classification['structural']['st_abnormality']['inferior'].add_matching_string('ST and T wave abnormality consider inferior ischemia')
classification['structural']['st_abnormality']['inferior'].add_matching_string('ST and T wave abnormality consider inferolateral ischemia')
classification['structural']['st_abnormality']['anterior'].add_matching_string('ST abnormality consider anterior ischemia')
classification['structural']['st_abnormality']['anterior'].add_matching_string('ST abnormality consider anterolateral ischemia')
classification['structural']['st_abnormality']['anterior'].add_matching_string('ST and T wave abnormality consider anterior ischemia')
classification['structural']['st_abnormality']['anterior'].add_matching_string('ST and T wave abnormality consider anterolateral ischemia')
classification['structural']['st_abnormality']['posterior'].add_matching_string('ST abnormality consider posterior ischemia')
classification['structural']['st_abnormality']['posterior'].add_matching_string('ST and T wave abnormality consider posterior ischemia')
classification['structural']['st_abnormality']['unspec'].add_matching_string('ST and T wave abnormality')
classification['structural']['st_abnormality']['unspec'].add_matching_string('ST abnormality')
classification['structural']['st_abnormality'].update_children_df(df)
classification['structural']['st_abnormality'].make_child_exclusive('unspec', df)
print(classification['structural']['st_abnormality'])
print(classification['structural']['st_abnormality']['septal'])
print(classification['structural']['st_abnormality']['lateral'])
print(classification['structural']['st_abnormality']['inferior'])
print(classification['structural']['st_abnormality']['anterior'])
print(classification['structural']['st_abnormality']['posterior'])
print(classification['structural']['st_abnormality']['unspec'])

In [ ]:
tw_abnormal = ecg_class('tw_abnormality', 'T wave abnormality')
classification['structural'].add_child(tw_abnormal)
tw_abnormal_septal = ecg_class('septal', 'T wave abnormality localized septally')
tw_abnormal_lateral = ecg_class('lateral', 'T wave abnormality localized laterally')
tw_abnormal_inferior = ecg_class('inferior', 'T wave abnormality localized inferiorly')
tw_abnormal_anterior = ecg_class('anterior', 'T wave abnormality localized anteriorly')
tw_abnormal_posterior = ecg_class('posterior', 'T wave abnormality localized posteriorly')
tw_abnormal_unspec = ecg_class('unspec', 'Unspecified T wave segment abnormality')
classification['structural']['tw_abnormality'].add_child(tw_abnormal_septal)
classification['structural']['tw_abnormality'].add_child(tw_abnormal_lateral)
classification['structural']['tw_abnormality'].add_child(tw_abnormal_inferior)
classification['structural']['tw_abnormality'].add_child(tw_abnormal_anterior)
classification['structural']['tw_abnormality'].add_child(tw_abnormal_posterior)
classification['structural']['tw_abnormality'].add_child(tw_abnormal_unspec)
classification['structural']['tw_abnormality']['septal'].add_matching_string('T wave abnormality consider septal ischemia')
classification['structural']['tw_abnormality']['septal'].add_matching_string('ST and T wave abnormality consider septal ischemia')
classification['structural']['tw_abnormality']['lateral'].add_matching_string('T wave abnormality consider lateral ischemia')
classification['structural']['tw_abnormality']['lateral'].add_matching_string('T wave abnormality consider anterolateral ischemia')
classification['structural']['tw_abnormality']['lateral'].add_matching_string('T wave abnormality consider inferolateral ischemia')
classification['structural']['tw_abnormality']['lateral'].add_matching_string('ST and T wave abnormality consider lateral ischemia')
classification['structural']['tw_abnormality']['lateral'].add_matching_string('ST and T wave abnormality consider anterolateral ischemia')
classification['structural']['tw_abnormality']['lateral'].add_matching_string('ST and T wave abnormality consider inferolateral ischemia')
classification['structural']['tw_abnormality']['inferior'].add_matching_string('T wave abnormality consider inferior ischemia')
classification['structural']['tw_abnormality']['inferior'].add_matching_string('T wave abnormality consider inferolateral ischemia')
classification['structural']['tw_abnormality']['inferior'].add_matching_string('ST and T wave abnormality consider inferior ischemia')
classification['structural']['tw_abnormality']['inferior'].add_matching_string('ST and T wave abnormality consider inferolateral ischemia')
classification['structural']['tw_abnormality']['anterior'].add_matching_string('T wave abnormality consider anterior ischemia')
classification['structural']['tw_abnormality']['anterior'].add_matching_string('T wave abnormality consider anterolateral ischemia')
classification['structural']['tw_abnormality']['anterior'].add_matching_string('ST and T wave abnormality consider anterior ischemia')
classification['structural']['tw_abnormality']['anterior'].add_matching_string('ST and T wave abnormality consider anterolateral ischemia')
classification['structural']['tw_abnormality']['posterior'].add_matching_string('T wave abnormality consider posterior ischemia')
classification['structural']['tw_abnormality']['posterior'].add_matching_string('ST and T wave abnormality consider posterior ischemia')
classification['structural']['tw_abnormality']['unspec'].add_matching_string('ST and T wave abnormality')
classification['structural']['tw_abnormality']['unspec'].add_matching_string('T wave abnormality')
classification['structural']['tw_abnormality'].update_children_df(df)
classification['structural']['tw_abnormality'].make_child_exclusive('unspec', df)
print(classification['structural']['tw_abnormality'])
print(classification['structural']['tw_abnormality']['septal'])
print(classification['structural']['tw_abnormality']['lateral'])
print(classification['structural']['tw_abnormality']['inferior'])
print(classification['structural']['tw_abnormality']['anterior'])
print(classification['structural']['tw_abnormality']['posterior'])
print(classification['structural']['tw_abnormality']['unspec'])

In [ ]:
st_elevation = ecg_class('st_elevation', 'ST elevation')
classification['structural'].add_child(st_elevation)
classification['structural']['st_elevation'].add_matching_string('ST elevation')
classification['structural']['st_elevation'].update_class_df(df)
print(classification['structural']['st_elevation'])

In [ ]:
lvh = ecg_class('lvh', 'Left ventricular hypertrophy')
rvh = ecg_class('rvh', 'Right ventricular hypertrophy')
classification['structural'].add_child(lvh)
classification['structural'].add_child(rvh)
classification['structural']['lvh'].add_matching_string('[Ll][Vv][Hh]')
classification['structural']['lvh'].add_matching_string('[Ll]eft ventricular hypertrophy')
classification['structural']['lvh'].update_class_df(df)
classification['structural']['rvh'].add_matching_string('[Rr][Vv][Hh]')
classification['structural']['rvh'].add_matching_string('[Rr]ight ventricular hypertrophy')
classification['structural']['rvh'].update_class_df(df)
print(classification['structural']['lvh'])
print(classification['structural']['rvh'])

In [ ]:
early_rep = ecg_class('early_rep', 'Early repolarization')
classification['structural'].add_child(early_rep)
classification['structural']['early_rep'].add_matching_string('[eE]arly repolarization')
classification['structural']['early_rep'].update_class_df(df)
print(classification['structural']['early_rep'])

In [ ]:
brugada = ecg_class('brugada', 'Brugada pattern')
digitalis = ecg_class('digitalis', 'Digitalis effect')
pericarditis = ecg_class('pericarditis', 'Acute pericarditis')
classification['structural'].add_child(brugada)
classification['structural'].add_child(digitalis)
classification['structural'].add_child(pericarditis)
classification['structural']['brugada'].add_matching_string('brugada')
classification['structural']['digitalis'].add_matching_string('digitalis')
classification['structural']['pericarditis'].add_matching_string('[Aa]cute pericarditis')
classification['structural']['brugada'].update_class_df(df)
classification['structural']['digitalis'].update_class_df(df)
classification['structural']['pericarditis'].update_class_df(df)
print(classification['structural']['brugada'])
print(classification['structural']['digitalis'])
print(classification['structural']['pericarditis'])

In [ ]:
counterclockwise_rotation = ecg_class('counterclockwise', 'Counter clockwise rotation')
classification['structural'].add_child(counterclockwise_rotation)
classification['structural']['counterclockwise'].add_matching_string('early transition')
classification['structural']['counterclockwise'].update_class_df(df)
print(classification['structural']['counterclockwise'])

In [ ]:
pulmonary_disease = ecg_class('pulmonary', 'Pulmonary disease pattern')
ventricular_preexcitation = ecg_class('ventricular_preexcitation', 'Ventricular pre-excitation')
classification['structural'].add_child(pulmonary_disease)
classification['structural'].add_child(ventricular_preexcitation)
classification['structural']['pulmonary'].add_matching_string('[pP]ulmonary disease')
classification['structural']['ventricular_preexcitation'].add_matching_string('[vV]entricular pre-excitation')
classification['structural']['ventricular_preexcitation'].add_matching_string('Wolff-Parkinson-White')
classification['structural']['ventricular_preexcitation'].add_matching_string('WPW')
classification['structural']['pulmonary'].update_class_df(df)
classification['structural']['ventricular_preexcitation'].update_class_df(df)
print(classification['structural']['pulmonary'])
print(classification['structural']['ventricular_preexcitation'])